This IPython notebook can load in a csv file containing a color map (expecting columns of scalars, RGB_r, RGB_g, and RGB_b) and resample it for various purposes.

Put the base name of the csv file here.

In [86]:
name = 'CoolWarmExtended'

In [87]:
import pandas
import numpy
import toyplot
import toyplot.pdf

from colormath.color_objects import *
from colormath.color_conversions import convert_color

In [88]:
data = pandas.read_csv(name + '.csv')
data

,scalar,RGB_r,RGB_g,RGB_b
0,0.0000,0.019608,0.188235,0.380392
1,0.0625,0.088502,0.321111,0.564935
2,0.1250,0.163394,0.444984,0.697505
3,0.1875,0.247059,0.555703,0.754101
4,0.2500,0.420691,0.676432,0.818692
5,0.3125,0.606455,0.789776,0.880278
6,0.3750,0.761471,0.868513,0.924559
7,0.4375,0.878050,0.925719,0.951949
8,0.5000,0.969085,0.966476,0.964935
9,0.5625,0.983856,0.897581,0.846830


Make a function that will take a scalar value (in the range 0 and 1) and return the appropriate RGB triple using interpolation in LAB space.

In [89]:
def black_body_lookup_upscaled(x):
    for index in xrange(0, data.index.size-1):
        low_scalar = data['scalar'][index]
        high_scalar = data['scalar'][index+1]
        if (x > high_scalar):
            continue
        low_rgb = sRGBColor(data['RGB_r'][index],
                            data['RGB_g'][index],
                            data['RGB_b'][index])
        high_rgb = sRGBColor(data['RGB_r'][index+1],
                             data['RGB_g'][index+1],
                             data['RGB_b'][index+1])
        low_lab = convert_color(low_rgb, LabColor)
        high_lab = convert_color(high_rgb, LabColor)
        interp = (x-low_scalar)/(high_scalar-low_scalar)
        mid_lab = LabColor(interp*(high_lab.lab_l-low_lab.lab_l) + low_lab.lab_l,
                           interp*(high_lab.lab_a-low_lab.lab_a) + low_lab.lab_a,
                           interp*(high_lab.lab_b-low_lab.lab_b) + low_lab.lab_b)
        return convert_color(mid_lab, sRGBColor).get_upscaled_value_tuple()
    raise ValueError, 'Scalar %f out of range' % x

def black_body_lookup(x):
    for index in xrange(0, data.index.size-1):
        low_scalar = data['scalar'][index]
        high_scalar = data['scalar'][index+1]
        if (x > high_scalar):
            continue
        low_rgb = sRGBColor(data['RGB_r'][index],
                            data['RGB_g'][index],
                            data['RGB_b'][index])
        high_rgb = sRGBColor(data['RGB_r'][index+1],
                             data['RGB_g'][index+1],
                             data['RGB_b'][index+1])
        low_lab = convert_color(low_rgb, LabColor)
        high_lab = convert_color(high_rgb, LabColor)
        interp = (x-low_scalar)/(high_scalar-low_scalar)
        mid_lab = LabColor(interp*(high_lab.lab_l-low_lab.lab_l) + low_lab.lab_l,
                           interp*(high_lab.lab_a-low_lab.lab_a) + low_lab.lab_a,
                           interp*(high_lab.lab_b-low_lab.lab_b) + low_lab.lab_b)
        return convert_color(mid_lab, sRGBColor).get_value_tuple()
    raise ValueError, 'Scalar %f out of range' % x

Make a long table of colors for plotting.

In [90]:
data_long = pandas.DataFrame({'scalar': numpy.linspace(0.0, 1.0, num=1024)})
data_long['sRGB'] = data_long['scalar'].apply(black_body_lookup)

In [92]:
palette = toyplot.color.Palette(colors=data_long['sRGB'].values)
colormap = toyplot.color.LinearMap(palette=palette)

In [93]:
canvas = toyplot.Canvas(width=512, height=200)
axes = canvas.axes()
axes.scatterplot(data_long['scalar'], numpy.zeros(data_long['scalar'].shape),
                 marker='|', size=500,
                 color=colormap)
axes.y.show = False
axes.x.show = False

In [94]:
toyplot.pdf.render(canvas, name + '.pdf')

Just out of curiosity, plot the luminance of the color map.

In [95]:
srgb_colors = data.apply(lambda row: sRGBColor(row['RGB_r'], row['RGB_g'], row['RGB_b']), axis=1)
lab_colors = srgb_colors.apply(lambda color: convert_color(color, LabColor))
luminance = lab_colors.apply(lambda lab: lab.lab_l)

canvas = toyplot.plot(data['scalar'], luminance)